In [10]:
from z3 import *

Here we simplify the problem from configuration to the bare minimum,
let a be the symbolic input, can fall into one of two ranges.

And c be one of the fields in the configuration file.

In [11]:
# Declare variables

# Here a represents a symbolic input.
a = Int('a')

# Here c represents a field in the configuration to be explained.
c = Int('c')

In [15]:
# Simplified route policy encoding: a nested if-then-else statements, only if no matches then it returns false.
# Note that the conditions here are one-to-one cosreponding to the input constraints.
route_policy = If(And(c <= a, a <= 13), False,
                If(And(5 <= a, a <= 7), False,
                    True))


# Verification problem, for all `a' that satisfy the input constraints, the route policy always return false. 

input_constraints = Or(And(11 <= a, a <= 13), And(5 <= a, a <= 7))

formula1 = ForAll([a], 
                  Implies(input_constraints,
                          route_policy == False)
                )

In [16]:
# This is the expected subspec on c, given formula 1 above
formula2 =  c <= 11

In [17]:

formula1 = ForAll([a,c], 
                  Implies(input_constraints,
                          route_policy == False)
                )

formula2 =  ForAll([a,c],c <= 11)

In [18]:
# Create a solver
solver = Solver()
solver.add(Not(formula1 == formula2))

# Check satisfiability
result = solver.check()

# Print the result: if 'unsat', the formulas are equivalent, otherwise not
if result == unsat:
    print("The formulas are equivalent.")
else:
    model = solver.model()
    print("The formulas are not equivalent.", model)

The formulas are equivalent.


**Question**:

Given that they are equivalent, how can we automatically derive formula2 from formula1?

**Note**:
If the IP range of the input constraint has overlap, 
like 9<=a<=13 or 10<=a<=15, then the above two formula no longer equivalent.
In this case, we need to rewrite the input IP prefix ranges into disjoint
ranges.

# Case 2: Subspec on irrelavant parts of the configuration

In [6]:
path1 = If(And(11 <= a, a <= 13), False,
                If(And(5 <= a, a <= 7), False,
                    True))

# Here we ask the subspec for the parameter c here.
path2 = If(And(c <= a, a <= 23), False,
                If(And(25 <= a, a <= 27), False,
                    True))

# Here path2 will never be executed, because path1 always return False given input_constraints.
policy2 = If(path1 == False, False, path2)

formula3 = ForAll([a], Implies(input_constraints, policy2 == False))

# So the subspec on c is empty, or simply True
subspec = True

In [7]:
# Create a solver
solver = Solver()
solver.add(Not(formula3 == subspec))

# Check satisfiability
result = solver.check()

# Print the result: if 'unsat', the formulas are equivalent, otherwise not
if result == unsat:
    print("The formulas are equivalent.")
else:
    model = solver.model()
    print("The formulas are not equivalent.", model)

The formulas are equivalent.


# Case 3: Subspec given an incorrect configuration?

In [9]:
# Here the route policy is incorrect, it may return True on some case (10 < a <= 13)
# route_policy = If(And(c <= a, a <= 13), False,
route_policy = If(And(c <= a, a <= 10), False,
                If(And(5 <= a, a <= 7), False,
                    True))


# Verification problem, for all `a' that satisfy the input constraints, the route policy always return false. 

input_constraints = Or(And(11 <= a, a <= 13), And(5 <= a, a <= 7))

formula1 = ForAll([a], 
                  Implies(input_constraints,
                          route_policy == False)
                )

What should the subspec for c should be?

One viable option might be to just refuse genearting subspec when the verification tasks fails.
As given an incorrect configuration there may not exist a sensisble subspec for any given field.

Except the faulty place, its subspec will tell you how to fix the error.

# Next steps 

1. Scale to large scale, network-wide verification tasks.
   The current study is focused on one individual configuration file. 
   Show that this approach can scale to large scale tasks.
   
2. Vary explanation "hole" granularity. Now we focus on explaining one individual field,
   how about the subspec for the whole route policies? Or the whole router.
   
   Here we may need to introduce some higher level abstraction like an uninterpreted function,
   to represent these larger "holes".